In [ ]:
!apt-get install -y python3.10-venv

In [2]:
!python -m venv my_darts_env
!source my_darts_env/bin/activate

In [ ]:
!pip install darts catboost torch torchvision --no-cache-dir

import catboost
from darts.datasets import WeatherDataset
from darts.models import CatBoostModel

import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
import matplotlib.pyplot as plt
from google.colab import drive
import os
import torch
import darts
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import EarlyStopping
from torchmetrics import MeanAbsolutePercentageError
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time
import datetime
from darts.models import RandomForest
from darts.models import NBEATSModel
from darts.models import NHiTSModel
from darts.models import XGBModel


print(catboost.__version__)

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
os.getcwd()

In [6]:
path = "/content/gdrive/MyDrive/Artigo TFT/Dados/tucurui.csv"

def read_data(path):
  # reads data
  data = pd.read_csv(path, delimiter=';', decimal=',').dropna()

  #specify study range (past 3 years)
  data = data.iloc[-365*3:-1,:]

  # some formating
  current_date_format = "%d/%m/%Y"

  data['Data'] = pd.to_datetime(data['Data'], format=current_date_format)

  # create time series
  prec = data[['Data', 'UPH610010000']].copy()
  prec_ts = TimeSeries.from_dataframe(prec, time_col="Data", value_cols=['UPH610010000'],fill_missing_dates=False, freq='D')

  vazao= data[['Data', 'VazaoNatural']].copy()
  vazao_ts = TimeSeries.from_dataframe(vazao, time_col="Data", value_cols=['VazaoNatural'],fill_missing_dates=True, freq='D')

  return prec_ts, vazao_ts

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

def nash_sutcliffe_efficiency(y_true, y_pred):
    numerator = np.sum((y_true - y_pred) ** 2)
    denominator = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (numerator / denominator)

def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def split_data(train_test, train_val, prec_ts, vazao_ts):
  # build train, val, test sets for flow(target) and rain data
  flow_train_val, flow_test = vazao_ts.split_before(train_test)
  flow_train, flow_val = flow_train_val.split_before(train_val)

  prec_train_val, prec_test = prec_ts.split_before(train_test)
  prec_train, prec_val = prec_train_val.split_before(train_val)

  return flow_train, flow_test, flow_val, prec_train, prec_test, prec_val

def data_scaling():
    # data scaling
    #inicialize scaler on flow data sets
    transformer_flow = Scaler()

    #transform the data on flow sets
    trans_flow_train = transformer_flow.fit_transform(flow_train)
    trans_flow = transformer_flow.transform(vazao_ts)

    trans_flow_test = transformer_flow.transform(flow_test)
    trans_flow_val = transformer_flow.transform(flow_val)
    #inicialize scaler on rainfall data sets
    transformer_prec = Scaler()

    #transform the data on rainfall sets
    trans_prec_train = transformer_prec.fit_transform(prec_train)
    trans_prec = transformer_prec.transform(prec_ts)

    trans_prec_test = transformer_prec.transform(prec_test)
    trans_prec_val = transformer_prec.transform(prec_val)

    return transformer_flow, transformer_prec, trans_flow_train,trans_flow,trans_flow_test,trans_prec_train,trans_prec,trans_prec_test,trans_prec_val,trans_flow_val

def build_samples_df(flow_test):

    # Define the starting date
    start_date = datetime.datetime(2022, 12, 31)

    # Define the window size and stride
    window_size = 14
    stride = 1

    # Convert TimeSeries to a pandas DataFrame
    timeseries=flow_test
    timeseries_df = timeseries.pd_dataframe()

    # Generate the samples
    num_samples = len(timeseries_df) - window_size + 1
    samples = []
    for i in range(num_samples):
        sample_start = start_date + datetime.timedelta(days=i)
        sample_dates = pd.date_range(start=sample_start, periods=window_size)
        if set(sample_dates).issubset(timeseries_df.index):
            sample = timeseries_df.loc[sample_dates]
            samples.append(TimeSeries.from_dataframe(sample))

    df_samples = pd.DataFrame()


    for i in range(len(samples)):

      valores = (samples[i].values())


      date = samples[i].start_time()

      temp = pd.DataFrame(valores).T
      temp['Date'] = date
      temp.set_index('Date', inplace=True)

      df_samples = pd.concat([df_samples,temp])
    return df_samples, samples

def build_backtest_df(backtest, transformer_flow):
    df_backtest = pd.DataFrame()
    for i in range(len(backtest)):
      valores = transformer_flow.inverse_transform(backtest[i]).values()

      date = backtest[i].start_time()

      temp = pd.DataFrame(valores).T
      temp['Date'] = date
      temp.set_index('Date', inplace=True)

      df_backtest = pd.concat([df_backtest,temp])
    return df_backtest



prec_ts, vazao_ts = read_data(path)
flow_train, flow_test, flow_val, prec_train, prec_test, prec_val = split_data(0.8, 0.9,prec_ts, vazao_ts)
transformer_flow,transformer_prec, trans_flow_train,trans_flow,trans_flow_test,trans_prec_train,trans_prec,trans_prec_test,trans_prec_val,trans_flow_val = data_scaling()

df_samples, samples=build_samples_df(flow_test)


In [12]:
my_stopper = EarlyStopping(
    monitor= "val_loss",
    patience=50,
    min_delta=0.0001,
    mode='min'
)
pl_trainer_kwargs = {"callbacks": [my_stopper]}

configs = {
    'config 1': {
        "model": CatBoostModel(lags=30, output_chunk_length=14),
        "optimizer": " "
    },
    'config 2': {
        "model": XGBModel(lags=30, output_chunk_length=14),
        "optimizer": " "
    },
    'config 3': {
        "model": RandomForest(lags=30, output_chunk_length=14, n_estimators=100, criterion="absolute_error"),
        "optimizer": " "
    },
    'config 4': {
        "model": NBEATSModel(input_chunk_length=30, output_chunk_length=14,optimizer_cls =torch.optim.Adam, optimizer_kwargs={"lr": 0.0005}, n_epochs=15, activation='ReLU'),
        "optimizer": torch.optim.Adam
    },
    'config 5': {
        "model": NBEATSModel(input_chunk_length=30, output_chunk_length=14,
                             optimizer_cls=torch.optim.RMSprop, optimizer_kwargs={"lr": 0.0005}, n_epochs=15,
                             activation='ReLU'),
        "optimizer": torch.optim.RMSprop
    },
    'config 6': {
        "model": NBEATSModel(input_chunk_length=30, output_chunk_length=14,optimizer_cls =torch.optim.SGD, optimizer_kwargs={"lr": 0.0005}, n_epochs=15, activation='ReLU'),
        "optimizer": torch.optim.SGD
    },
    'coonfig 7': {
        "model": NHiTSModel(input_chunk_length=30, output_chunk_length=14,optimizer_cls =torch.optim.Adam, optimizer_kwargs={"lr": 0.0005}, n_epochs=15, activation='ReLU'),
        "optimizer": torch.optim.Adam
    },
    'config 8': {
        "model": NHiTSModel(input_chunk_length=30, output_chunk_length=14,
                           optimizer_cls=torch.optim.RMSprop, optimizer_kwargs={"lr": 0.0005}, n_epochs=15,
                           activation='ReLU'),
        "optimizer": torch.optim.RMSprop
    },
    'config 9': {
        "model": NHiTSModel(input_chunk_length=30, output_chunk_length=14,optimizer_cls =torch.optim.SGD, optimizer_kwargs={"lr": 0.0005}, n_epochs=15, activation='ReLU'),
        "optimizer": torch.optim.SGD
    }
}

# Initialize an empty dictionary to store metrics for each model
metrics_dict = {}

In [ ]:

# Initialize an empty dictionary to store metrics for each model
metrics_dict = {}

# Iterate over each model in the configs dictionary
for model_name, config in configs.items():
    model = config["model"]

    # Fit the model
    inicio = time.time()
    model.fit(trans_flow_train, val_series=trans_flow_val)
    fim = time.time()
    tempo = fim - inicio

    # Perform backtesting
    backtest = model.historical_forecasts(series=trans_flow_test,
                                          forecast_horizon=14,
                                          stride=1,
                                          retrain=False,
                                          overlap_end=False,
                                          last_points_only=False,
                                          verbose=False)

    # Build backtest dataframe
    df_backtest = build_backtest_df(backtest, transformer_flow)

    # Initialize a list to store metrics for the current model
    metricas = []

    # Compute metrics for each forecast horizon
    for i in range(14):
        mape = mean_absolute_percentage_error(df_backtest.iloc[:, i], df_samples.iloc[:, i])
        smape = symmetric_mean_absolute_percentage_error(df_backtest.iloc[:, i], df_samples.iloc[:, i])
        mae = mean_absolute_error(df_backtest.iloc[:, i], df_samples.iloc[:, i])
        rmse = root_mean_squared_error(df_backtest.iloc[:, i], df_samples.iloc[:, i])
        metricas.append({"mape": mape, "smape": smape, "mae": mae, "rmse": rmse})

    # Save metrics and additional information for the current model configuration in the dictionary
    metrics_dict[model_name] = {
        "metrics": metricas,
        # "optimizer": config["optimizer"].__name__ if config["optimizer"] else "None",
        "optimizer": config["optimizer"].__name__ if not isinstance(config["optimizer"], str) else config["optimizer"],
        "fit_time": tempo,
        "model_name": type(model).__name__[:-5]
    }


In [14]:
# Create a DataFrame from the metrics dictionary
df_metrics = pd.DataFrame(metrics_dict).T

# Calculate the average of each metric
df_metrics["MAPE"] = df_metrics["metrics"].apply(lambda x: round(sum(metric["mape"] for metric in x) / len(x),2))
df_metrics["SMAPE"] = df_metrics["metrics"].apply(lambda x: round(sum(metric["smape"] for metric in x) / len(x),2))
df_metrics["MAE"] = df_metrics["metrics"].apply(lambda x: round(sum(metric["mae"] for metric in x) / len(x),2))
df_metrics["RMSE"] = df_metrics["metrics"].apply(lambda x: round(sum(metric["rmse"] for metric in x) / len(x),2))

# Drop the "metrics" column as it's no longer needed
df_metrics.drop(columns=["metrics"], inplace=True)
df_metrics = df_metrics.set_index('model_name')

# Display the resulting DataFrame
df_metrics

,optimizer,fit_time,MAPE,SMAPE,MAE,RMSE
model_name,,,,,,
CatBoost,,132.791299,16.77,16.35,1923.04,2563.72
XGB,,37.193987,19.66,16.91,1985.44,2722.06
RandomF,,103.911387,10.59,10.92,1331.48,1819.65
NBEATS,Adam,107.442902,17.42,19.50,2162.25,2618.32
NBEATS,RMSprop,91.328767,38.36,22.00,2158.95,2676.95
NBEATS,SGD,63.42352,604.86,112.29,9277.72,10304.94
NHiTS,Adam,25.431513,9.80,9.34,1075.59,1705.04
NHiTS,RMSprop,22.673881,31.92,25.06,2343.86,2921.23
NHiTS,SGD,17.762575,50.40,45.57,4524.50,5223.57
